# Topic Modeling -- NMF

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import gensim
import time
import time

import nltk
import stanza
import gensim
import re

#topic modeling
from sklearn.decomposition import NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression as lm

import matplotlib.collections as mcol
from matplotlib.legend_handler import HandlerLineCollection, HandlerTuple
from matplotlib.lines import Line2D

import seaborn as sns

In [3]:
df = pd.read_pickle("/home/zz3hs/git/dspg21oss/data/dspg21oss/readme_lemma_try.pkl")

In [4]:
df

,index,slug,readme_text,batch,as_of,status,readme_lemma
1,1,Distrotech/mysql-server,MySQL Server 5.6\r\n\r\nThis is a release of M...,oss_readme_batch1_1,6/11/21 17:21,Done,"[MySQL, server, 5.6, release, MySQL, dual, lic..."
2,2,DaichiUeura/Emacs-for-Windows--xj-,"Copyright (C) 2001, 2002, 2003, 2004, 2005, 20...",oss_readme_batch1_1,6/11/21 17:21,Done,"[Copyright, C, 2001, 2002, 2003, 2004, 2005, 2..."
3,3,httpgit12/jb4evea-16,"<img alt=""Swift logo"" height=""70"" src=""https:/...",oss_readme_batch1_1,6/11/21 17:21,Done,"[img, alt=, Swift, logo, height, 70, src=, htt..."
4,4,Tehsurfer/hugo-contrarian,# Contrarian website\nThis website is the face...,oss_readme_batch1_1,6/11/21 17:21,Done,"[Contrarian, website, website, face, the-arrrb..."
5,5,jrockway/emacs,"Copyright (C) 2001, 2002, 2003, 2004, 2005, 20...",oss_readme_batch1_1,6/11/21 17:21,Done,"[Copyright, C, 2001, 2002, 2003, 2004, 2005, 2..."
...,...,...,...,...,...,...,...
96,96,cjamieso/moodle381,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done,"[Moodle, world, open, source, learning, platfo..."
97,97,Marlo81/meimudl.at,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done,"[Moodle, world, open, source, learning, platfo..."
98,98,leoauri/street-college-moodle-core,.-..-.\n __...,oss_readme_batch1_1,6/11/21 17:21,Done,"[Moodle, world, open, source, learning, platfo..."
99,99,moodleworkplace/multitenancy,This is a repository FOR TESTING PURPOSES ONLY...,oss_readme_batch1_1,6/11/21 17:21,Done,"[repository, testing, purpose, only, contain, ..."


In [5]:
# function slightly modified from https://nlpforhackers.io/topic-modeling/

def print_topics(topic_term_mat, vectorizer, top_n=10):
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
        print("\nTopic %d:" % (idx))
        #print([(vectorizer.get_feature_names()[i], topic[i])  # printing out words corresponding to indices found in next line
                        #for i in topic.argsort()[:-top_n - 1:-1]])  # finding indices of top words in topic
            
        print_list = [(vectorizer.get_feature_names()[i], topic[i])  
                        for i in topic.argsort()[:-top_n - 1:-1]]
        for item in print_list:
            print(item)
            
# Function to format topics as a "list of list of strings".
# Needed for topic coherence function in Gensim

# function modified from https://nlpforhackers.io/topic-modeling/

def str_topics(topic_term_mat, vectorizer, top_n=10):

    #input. top_n: how many words to list per topic.  If -1, then list all words.
       
    topic_words = []
    
    for idx, topic in enumerate(topic_term_mat):  # loop through each row of H.  idx = row index.  topic = actual row
            
        if top_n == -1:   
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[::-1]])
        else:
            topic_words.append([vectorizer.get_feature_names()[i] for i in topic.argsort()[:-top_n - 1:-1]])
    
    str_wds = []
    
    for wds in topic_words:
        str_wds.append(", ".join(wds))    
    
    return str_wds 

In [6]:
# input needed for LDA, NMF (all from Scikit-Learn) is one string per document (not a list of strings)
text = []
docs = df["readme_lemma"]

for abstract in docs:
    text.append(" ".join(abstract))

In [7]:
tfidf_vectorizer = TfidfVectorizer(max_df=1.0, min_df=3, lowercase=False)
tf_idf = tfidf_vectorizer.fit_transform(text)

In [8]:
tfidf_vectorizer

TfidfVectorizer(lowercase=False, min_df=3)

In [21]:
# create model

num_topics = 10

t1 = time.time()
nmf_model = NMF(n_components=num_topics, random_state = 1)
doc_topic = nmf_model.fit_transform(tf_idf)
t2 = time.time()
print(f"  Model time: {t2-t1}")

topic_term = nmf_model.components_

  Model time: 0.03133845329284668


In [22]:
nmf_model

NMF(n_components=10, random_state=1)

In [23]:
topic_frame = pd.DataFrame(doc_topic, columns=["Topic"+" "+str(i) for i in range(num_topics)])
topic_frame

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9
0,0.000000,0.000000,0.463334,0.000097,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.531464,0.000000,0.000373,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.008914,0.006652,0.002325,0.000000,0.766971,0.000000,0.020202
3,0.000000,0.000000,0.000000,0.039463,0.027039,0.047962,0.024066,0.013454,0.064567,0.517476
4,0.000000,0.531530,0.000000,0.000365,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...
95,0.000000,0.000000,0.000000,0.548272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
96,0.000000,0.000000,0.000000,0.548272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
97,0.000000,0.000000,0.000000,0.548272,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
98,0.000536,0.003513,0.000000,0.162487,0.000000,0.000000,0.051212,0.000000,0.551915,0.000000


In [24]:
topic_words = str_topics(topic_term, tfidf_vectorizer, top_n=10)
topic_words

['samba, Samba, list, org, mailing, lists, include, window, http, Linux',
 'Emacs, hold, file, code, GNU, bug, configure, script, gnu, install',
 'MySQL, mysql, license, GPL, GPLv2, version, http, FOSS, exception, choice',
 'Moodle, moodle, https, learning, partner, org, world, platform, provide, open',
 'phpmyadmin, https, target, net, image, svg, ci, badge, browserstack, branch',
 'SaltStack, saltstack, salt, https, Salt, com, training, Open, http, issue',
 'Homebrew, homebrew, brew, cask, md, atom, https, github, core, error',
 'swift, job, ci, string, linux, https, build, icon, 04, x86_64',
 'plugin, suite, build, zip, tool, core, run, directory, lightweight, JDK',
 'github, https, com, wiki, 2016, 10, performance, 14, in, 15']